# Load shipment orders

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date,time,datetime,timedelta

import warnings 
warnings.filterwarnings("ignore")

data = pd.read_csv('./shipbill_lonlat_180days_20181208142900.csv',parse_dates=['dt','first_rev_tm','last_rev_tm','succ_delv_tm','redelv_tm'],low_memory=False)
data.head()

,dt,dp,ship_bill_cate_cd,trader_mold,shipping_bill_id,waybill_state_enum,sale_ord_id,create_ord_dt,delv_center_num,sta_id,...,last_exam_tm,first_rev_tm,last_rev_tm,rev_confirm_tm,succ_delv_tm,rejection_tm,shelves_tm,redelv_tm,lat,lng
0,2018-06-14,HISTORY,0,1001.0,76278252822,150,76278252822,2018-06-11,6.0,1380,...,2018-06-11 14:42:33.0,2018-06-11 17:08:27,2018-06-11 17:08:27,2018-06-11 15:38:41.0,2018-06-11 18:57:21,NaN,NaN,NaT,39.80117,116.577850
1,2018-06-14,HISTORY,0,1001.0,76280295920,150,76280295920,2018-06-11,6.0,603,...,2018-06-11 14:42:36.0,2018-06-11 15:30:44,2018-06-11 15:30:44,2018-06-11 15:18:02.0,2018-06-11 16:44:12,NaN,NaN,NaT,39.78768,116.515550
2,2018-06-14,HISTORY,0,1001.0,76280345021,150,76280345021,2018-06-11,6.0,1380,...,NaN,NaT,NaT,2018-06-11 15:29:42.0,2018-06-11 16:10:50,NaN,NaN,NaT,39.79510,116.586460
3,2018-06-14,HISTORY,0,1001.0,76280630046,150,76280630046,2018-06-11,6.0,1380,...,2018-06-11 14:34:02.0,2018-06-11 15:35:34,2018-06-11 15:35:34,2018-06-11 15:35:34.0,2018-06-11 19:06:45,NaN,NaN,NaT,39.80064,116.565948
4,2018-06-14,HISTORY,0,1001.0,76281269919,150,76281269919,2018-06-11,6.0,1380,...,2018-06-11 15:25:47.0,2018-06-11 15:58:26,2018-06-11 15:58:26,2018-06-11 15:31:59.0,2018-06-11 18:53:24,NaN,NaN,NaT,39.80644,116.576730


In [2]:
data.shape

(781717, 39)

In [3]:
dt = data[['sta_id','shipping_bill_id','last_rev_operr_id','lng','lat','last_rev_tm','succ_delv_tm']].dropna()
dt['last_rev_operr_id'] = dt.last_rev_operr_id.astype(int)
dt['rev_date'] = [d.date() for d in dt['last_rev_tm']]
dt['delv_date'] = [d.date() for d in dt['succ_delv_tm']]

dt.columns = ['station_id','order_id','courier_id','lng','lat','rev_tm','delv_tm','rev_date','delv_date']

#dt['succ_delv_date'] = [d.date() for d in dt['succ_delv_tm']]
#dt['succ_delv_time'] = [d.time() for d in dt['succ_delv_tm']]

dt['rev_date'] = pd.to_datetime(dt['rev_date'])
dt['delv_date'] = pd.to_datetime(dt['delv_date'])

dt['batch_id'] = -1
dt['start_tm'] = -1
#dt['tw_ed'] = -1
dt['end_tm'] = -1
#dt['tw_ed_95'] = -1
dt.head()

,station_id,order_id,courier_id,lng,lat,rev_tm,delv_tm,rev_date,delv_date,batch_id,start_tm,end_tm
0,1380,76278252822,20604802,116.577850,39.80117,2018-06-11 17:08:27,2018-06-11 18:57:21,2018-06-11,2018-06-11,-1,-1,-1
1,603,76280295920,20601988,116.515550,39.78768,2018-06-11 15:30:44,2018-06-11 16:44:12,2018-06-11,2018-06-11,-1,-1,-1
3,1380,76280630046,231387,116.565948,39.80064,2018-06-11 15:35:34,2018-06-11 19:06:45,2018-06-11,2018-06-11,-1,-1,-1
4,1380,76281269919,20563342,116.576730,39.80644,2018-06-11 15:58:26,2018-06-11 18:53:24,2018-06-11,2018-06-11,-1,-1,-1
5,603,76281327199,29037,116.515590,39.79997,2018-06-11 15:11:29,2018-06-11 16:30:16,2018-06-11,2018-06-11,-1,-1,-1


In [4]:
# remove data for 6/11/2018 since it contains half day data
dt = dt[dt['rev_date']!=date(2018,6,11)]
dt.head()

,station_id,order_id,courier_id,lng,lat,rev_tm,delv_tm,rev_date,delv_date,batch_id,start_tm,end_tm
648,603,76283518260,77562,116.517190,39.785620,2018-06-12 07:46:56,2018-06-11 17:56:14,2018-06-12,2018-06-11,-1,-1,-1
1495,1380,76275274875,20613701,116.565930,39.795370,2018-06-12 08:33:43,2018-06-12 12:35:23,2018-06-12,2018-06-12,-1,-1,-1
1496,603,76281645977,20063521,116.514780,39.808490,2018-06-12 07:51:14,2018-06-12 09:16:30,2018-06-12,2018-06-12,-1,-1,-1
1497,1380,76283979376,20456527,116.581110,39.800550,2018-06-12 15:52:28,2018-06-12 19:22:42,2018-06-12,2018-06-12,-1,-1,-1
1498,1380,76285157651,20218256,116.584033,39.803021,2018-06-12 07:39:49,2018-06-12 08:02:17,2018-06-12,2018-06-12,-1,-1,-1


In [5]:
dt['rev_date'].dtype

dtype('<M8[ns]')

In [6]:
# select frequent courier men
tot_sta = dt.groupby(['station_id'])['order_id'].count().reset_index(name='tot')
tot_sta['threshold'] = (tot_sta.tot/55).astype(int)
tot_per = dt.groupby(['station_id','courier_id'])['order_id'].count().reset_index(name='tot_courier')
sel_operr = pd.merge(tot_per, tot_sta, on='station_id', how='outer')
sel_operr = sel_operr[sel_operr.tot_courier>sel_operr.threshold]
sel_operr = list(set(sel_operr.courier_id))
len(sel_operr)

42

In [7]:
dt = dt[dt['courier_id'].isin(sel_operr)].reset_index(drop=True)
dt.head()  # 781717 to 695327; 11% reduction

,station_id,order_id,courier_id,lng,lat,rev_tm,delv_tm,rev_date,delv_date,batch_id,start_tm,end_tm
0,603,76283518260,77562,116.517190,39.785620,2018-06-12 07:46:56,2018-06-11 17:56:14,2018-06-12,2018-06-11,-1,-1,-1
1,1380,76283979376,20456527,116.581110,39.800550,2018-06-12 15:52:28,2018-06-12 19:22:42,2018-06-12,2018-06-12,-1,-1,-1
2,1380,76285157651,20218256,116.584033,39.803021,2018-06-12 07:39:49,2018-06-12 08:02:17,2018-06-12,2018-06-12,-1,-1,-1
3,603,76285230809,20457191,116.503630,39.805910,2018-06-12 07:43:02,2018-06-12 10:36:22,2018-06-12,2018-06-12,-1,-1,-1
4,603,76285245266,29037,116.515590,39.799970,2018-06-12 15:00:09,2018-06-12 16:25:21,2018-06-12,2018-06-12,-1,-1,-1


In [8]:
dt.shape

(694275, 12)

# Determine sub-batch labels and time windows

In [9]:
# make sure delv_tm >= rev_tm; sanity check
dt = dt[dt['rev_tm']<dt['delv_tm']]
dt.shape

(693472, 12)

In [10]:
# ignore all records that are associated with orders delivered the next day or later
dt = dt[dt['delv_date']==dt['rev_date']].reset_index(drop=True)
dt.head()

,station_id,order_id,courier_id,lng,lat,rev_tm,delv_tm,rev_date,delv_date,batch_id,start_tm,end_tm
0,1380,76283979376,20456527,116.581110,39.800550,2018-06-12 15:52:28,2018-06-12 19:22:42,2018-06-12,2018-06-12,-1,-1,-1
1,1380,76285157651,20218256,116.584033,39.803021,2018-06-12 07:39:49,2018-06-12 08:02:17,2018-06-12,2018-06-12,-1,-1,-1
2,603,76285230809,20457191,116.503630,39.805910,2018-06-12 07:43:02,2018-06-12 10:36:22,2018-06-12,2018-06-12,-1,-1,-1
3,603,76285245266,29037,116.515590,39.799970,2018-06-12 15:00:09,2018-06-12 16:25:21,2018-06-12,2018-06-12,-1,-1,-1
4,1380,76285314556,20163882,116.570290,39.794434,2018-06-12 07:29:07,2018-06-12 08:39:41,2018-06-12,2018-06-12,-1,-1,-1


In [11]:
dt.shape

(681293, 12)

In [12]:
grp_dt = dt.groupby(['station_id','courier_id','rev_date'])

In [13]:
def assignBatch(df,gap):  
# input: grouped bills for (sta, operr, date); gap of consecutive batches in minutes
# output:
       # 1. batch
       # 2. tw_st: time window start point for this batch; the lastest 'rev_tm' of all bills in this batch
       # 3. tw_ed: time window end point for this batch; the latest 'delv_tm' of all bills in this batch
    
    df2 = df
    df2 = df2.sort_values(by=['rev_tm'])  # make sure all timestamps are of format 'datetime'    
    
    #st_tm = df2['rev_tm'].iloc[0]
    
    #start = st_tm
    label = 0
    bat = []
    #win_st = []
    lookup_start = {}
    lookup_end = {}
    lookup_end_90 = {}
    #lookup_end_95 = {}
    ed_set = []
    for i in range(df2.shape[0]):
        if i == 0:
            bat.append(label)
            start = df2['rev_tm'].iloc[i]
            #win_st.append(tw)
            last = df2['delv_tm'].iloc[i]
            ed_set.append(df2['delv_tm'].iloc[i])
            
        elif df2['rev_tm'].iloc[i]-df2['rev_tm'].iloc[i-1]<timedelta(minutes=gap):
            bat.append(label)
            #win_st.append(tw)
            start = df2['rev_tm'].iloc[i]
            ed_set.append(df2['delv_tm'].iloc[i])
            
            if df2['delv_tm'].iloc[i]>last:
                last = df2['delv_tm'].iloc[i]
                
        elif df2['rev_tm'].iloc[i]-df2['rev_tm'].iloc[i-1]>=timedelta(minutes=gap):
            if len(ed_set)==1 or i==df2.shape[0]-1:
                bat.append(np.nan)
                lookup_start[np.nan] = np.nan
                lookup_end_90[np.nan] = np.nan
            else:
                lookup_start[label] = start
                lookup_end[label] = last
                
                lookup_end_90[label] = pd.Series(ed_set).quantile(0.9)  #0.9
                #lookup_end_95[label] = pd.Series(ed_set).quantile(0.95)
                
                ed_set = []
                #tw = df2['rev_tm'].iloc[i]
                label += 1
                bat.append(label)
                #win_st.append(tw)
                start = df2['rev_tm'].iloc[i]
                last = df2['delv_tm'].iloc[i]
                
                ed_set.append(df2['delv_tm'].iloc[i])
        #elif i == df.shape[0]-1:
         #   lookup[label] = last
    lookup_start[label] = start
    lookup_end[label] = last
    lookup_end_90[label] = pd.Series(ed_set).quantile(0.9)  # 0.9
    #lookup_end_95[label] = pd.Series(ed_set).quantile(0.95)
    
    
    df2['batch_id'] = bat
    #df2['tw_st'] = win_st
    df2['start_tm'] = [lookup_start.get(df2['batch_id'].iloc[i]) for i in range(df2.shape[0])]
    #df2['tw_ed'] = [lookup_end.get(df2['batch'].iloc[i]) for i in range(df2.shape[0])]
    df2['end_tm'] = [lookup_end_90.get(df2['batch_id'].iloc[i]) for i in range(df2.shape[0])]
    #df2['tw_ed_95'] = [lookup_end_95.get(df2['batch'].iloc[i]) for i in range(df2.shape[0])]
    
    df2 = df2.dropna()
            
    return df2

In [14]:
res = grp_dt.apply(assignBatch,120)  # threshold 120 min, as of Jan 10
res.head()

station_id     order_id  courier_id  \
station_id courier_id rev_date                                               
603        6273       2018-06-12 911          603  76290723549        6273   
                                 847          603  76289656893        6273   
                                 1344         603  76296915894        6273   
                                 794          603  76288977913        6273   
                                 2766         603  76775222850        6273   

                                             lng       lat  \
station_id courier_id rev_date                               
603        6273       2018-06-12 911   116.49725  39.77230   
                                 847   116.50843  39.80340   
                                 1344  116.50846  39.80072   
                                 794   116.50846  39.80072   
                                 2766  116.50846  39.80072   

                                                   rev_tm             delv_tm  \
station_id courier_id rev_date                                                  
603        6273       2018-06-12 911  2018-06-12 07:38:58 2018-06-12 11:33:04   
                                 847  2018-06-12 07:39:06 2018-06-12 11:22:28   
                                 1344 2018-06-12 07:39:10 2018-06-12 09:14:30   
                                 794  2018-06-12 07:39:17 2018-06-12 09:34:05   
                                 2766 2018-06-12 07:39:25 2018-06-12 08:56:56   

                                        rev_date  delv_date  batch_id  \
station_id courier_id rev_date                                          
603        6273       2018-06-12 911  2018-06-12 2018-06-12       0.0   
                                 847  2018-06-12 2018-06-12       0.0   
                                 1344 2018-06-12 2018-06-12       0.0   
                                 794  2018-06-12 2018-06-12       0.0   
                                 2766 2018-06-12 2018-06-12       0.0   

                                                 start_tm  \
station_id courier_id rev_date                              
603        6273       2018-06-12 911  2018-06-12 07:57:52   
                                 847  2018-06-12 07:57:52   
                                 1344 2018-06-12 07:57:52   
                                 794  2018-06-12 07:57:52   
                                 2766 2018-06-12 07:57:52   

                                                       end_tm  
station_id courier_id rev_date                                 
603        6273       2018-06-12 911  2018-06-12 11:41:03.400  
                                 847  2018-06-12 11:41:03.400  
                                 1344 2018-06-12 11:41:03.400  
                                 794  2018-06-12 11:41:03.400  
                                 2766 2018-06-12 11:41:03.400

In [15]:
output = res.drop(['station_id', 'courier_id','rev_date','delv_date'], axis=1)
output.head()

order_id        lng       lat  \
station_id courier_id rev_date                                            
603        6273       2018-06-12 911   76290723549  116.49725  39.77230   
                                 847   76289656893  116.50843  39.80340   
                                 1344  76296915894  116.50846  39.80072   
                                 794   76288977913  116.50846  39.80072   
                                 2766  76775222850  116.50846  39.80072   

                                                   rev_tm             delv_tm  \
station_id courier_id rev_date                                                  
603        6273       2018-06-12 911  2018-06-12 07:38:58 2018-06-12 11:33:04   
                                 847  2018-06-12 07:39:06 2018-06-12 11:22:28   
                                 1344 2018-06-12 07:39:10 2018-06-12 09:14:30   
                                 794  2018-06-12 07:39:17 2018-06-12 09:34:05   
                                 2766 2018-06-12 07:39:25 2018-06-12 08:56:56   

                                       batch_id            start_tm  \
station_id courier_id rev_date                                        
603        6273       2018-06-12 911        0.0 2018-06-12 07:57:52   
                                 847        0.0 2018-06-12 07:57:52   
                                 1344       0.0 2018-06-12 07:57:52   
                                 794        0.0 2018-06-12 07:57:52   
                                 2766       0.0 2018-06-12 07:57:52   

                                                       end_tm  
station_id courier_id rev_date                                 
603        6273       2018-06-12 911  2018-06-12 11:41:03.400  
                                 847  2018-06-12 11:41:03.400  
                                 1344 2018-06-12 11:41:03.400  
                                 794  2018-06-12 11:41:03.400  
                                 2766 2018-06-12 11:41:03.400

In [16]:
max_bat_size = res.groupby(['station_id','courier_id','rev_date','batch_id'])['batch_id'].count().reset_index(name='batch_size')
max_per = max(max_bat_size['batch_size'])
max_per

299

In [17]:
output.shape

(679734, 8)

In [ ]:
output.to_csv('batch_info_120min.csv')

In [ ]:
res['batch_id'].max()

In [24]:
res['batch_id'].quantile(0.97)

2.0

In [25]:
batch4 = res[res['batch_id']==4]
batch4.head()

station_id       order_id  \
station_id courier_id rev_date                                       
1380       294301     2018-06-19 30065         1380    76923176106   
                                 27612         1380    76307342581   
                      2018-11-14 598510        1380  VE48411641261   
                                 598765        1380    81560882396   
                                 602302        1380    82104167394   
                                 598545        1380  VC49149984973   
                                 598905        1380    81591709170   
                                 598754        1380    81556610257   
                                 598560        1380    81583372944   
                                 599987        1380    81875963374   
                                 600066        1380    81909087502   
                                 600284        1380    81959077225   
                                 600294        1380    81960134504   
                                 600417        1380    81985741574   
           20039485   2018-09-22 404614        1380    78106702847   
                                 403909        1380    77997229937   
                                 406517        1380    79433905704   
                                 405318        1380    78143443792   
                                 403935        1380    78145767093   
                                 405217        1380    78141603070   
                                 404611        1380    78106517586   
                                 404587        1380    78091548498   
                                 404555        1380    78047160732   
                                 404615        1380    78106710679   
                                 404616        1380    78106815763   
                                 405115        1380    78139368853   
                                 406483        1380    79431909229   
           20039560   2018-11-12 592300        1380    81992795655   
                                 591664        1380    81956140968   
           20218256   2018-06-19 32392         1380    77003797228   
...                                             ...            ...   
                                 32244         1380    76997248073   
           20316432   2018-06-19 32373         1380    77002004833   
                                 31775         1380    76981976900   
                                 32260         1380    76997665356   
                                 32379         1380    77002356363   
                                 30044         1380    76919661608   
                                 29575         1380    76477106809   
                                 28591         1380    76455066940   
                                 32385         1380    77002966504   
                                 32682         1380  VB42784399560   
                                 30566         1380    76956092160   
                                 32344         1380    76999893251   
                                 29818         1380    76487181725   
                                 32333         1380    76999435752   
                                 29679         1380    76481259536   
                                 32386         1380    77002987201   
                                 32384         1380    77002697378   
                                 32339         1380    76999604386   
                                 27632         1380    76393137785   
                                 31821         1380    76982996358   
                                 31338         1380    76973449037   
                                 27235         1380  VB43580300116   
                                 29470         1380    76474069852   
                                 32306         1380    76998889227   
                                 32355         1380

In [26]:
sub = res[(res.station_id==603)&(res.courier_id==20497461)&(res.rev_date==date(2018,11,20))].reset_index(drop=True)
sub = sub.sort_values(['rev_tm'])
sub.head()

,station_id,order_id,courier_id,lng,lat,rev_tm,delv_tm,rev_date,delv_date,batch_id,start_tm,end_tm
0,603,82224898054,20497461,116.501373,39.786695,2018-11-20 07:51:09,2018-11-20 09:36:40,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
1,603,VC48574646466,20497461,116.501373,39.786695,2018-11-20 07:51:44,2018-11-20 11:44:59,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
2,603,VB48645196946,20497461,116.512940,39.794370,2018-11-20 07:53:05,2018-11-20 11:50:30,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
3,603,82222850275,20497461,116.501373,39.786695,2018-11-20 07:54:00,2018-11-20 10:22:54,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
4,603,82224105541,20497461,116.511467,39.792728,2018-11-20 07:54:18,2018-11-20 10:28:25,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
5,603,82224168104,20497461,116.513090,39.793820,2018-11-20 07:54:39,2018-11-20 11:42:47,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
6,603,82221923392,20497461,116.511467,39.792728,2018-11-20 07:54:51,2018-11-20 11:04:15,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
7,603,82223287235,20497461,116.501373,39.786695,2018-11-20 07:56:33,2018-11-20 09:36:40,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
8,603,81813304048,20497461,116.511467,39.792728,2018-11-20 07:56:45,2018-11-20 10:24:58,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600
9,603,82223915427,20497461,116.511467,39.792728,2018-11-20 07:57:33,2018-11-20 10:53:34,2018-11-20,2018-11-20,0.0,2018-11-20 08:13:56,2018-11-20 11:44:13.600


In [16]:
sub.to_csv('sample_batch_603_20497461.csv')